# Task 4.5.2: Advanced Streamlit - Multi-Page Apps & Performance

**Module:** 4.5 - Demo Building & Prototyping  
**Time:** 2-3 hours  
**Difficulty:** ⭐⭐⭐☆☆

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Build multi-page Streamlit applications
- [ ] Master session state for conversation and settings persistence
- [ ] Implement caching strategies for models and data
- [ ] Optimize performance with lazy loading and async operations
- [ ] Create polished, production-ready Streamlit apps

---

## 📚 Prerequisites

- Completed: Task 4.5.1 (Gradio Advanced) or equivalent Gradio knowledge
- Knowledge of: Python, basic understanding of web concepts
- Familiarity with: LLM APIs (for chat examples)

---

## 🌍 Real-World Context

Your team loves the quick prototype you made in Gradio. But now the Product Manager says:

*"We need a dashboard with multiple pages - analytics, chat history, admin settings. And it needs to remember user preferences. Can we do that?"*

Streamlit excels at exactly this. While Gradio is great for ML demos, Streamlit shines for:
- Multi-page data applications
- Complex state management
- Dashboard-style layouts
- Long-running data science workflows

---

## 🧒 ELI5: Gradio vs Streamlit

> **Gradio** is like a microwave - pop in your model, press a button, get results. Perfect for quick demos.
>
> **Streamlit** is like a full kitchen - more setup, but you can cook anything. Multiple burners (pages), a fridge for storage (session state), and you decide exactly how the kitchen looks.
>
> Both are great! Use Gradio for "look at my model" demos. Use Streamlit for "look at my complete application" demos.

---

## Important: How Streamlit Works Differently

Unlike Gradio (which uses callbacks), Streamlit **re-runs your entire script** from top to bottom every time anything changes. This has implications:

1. **Variables reset** - Use `st.session_state` to persist data
2. **Models reload** - Use `@st.cache_resource` to cache expensive objects
3. **Data recalculates** - Use `@st.cache_data` to cache computed results

Understanding this "rerun" model is key to writing efficient Streamlit apps!

---

## Part 1: Streamlit Basics Refresher

Since Streamlit runs as a standalone script (not in a notebook), we'll write our examples as Python files. Let's start with a basic refresher.

### Installing Streamlit

In [ ]:
# Install Streamlit
!pip install -q streamlit>=1.30.0

### Basic Streamlit App Structure

Here's the simplest Streamlit app:

In [ ]:
# Write a basic Streamlit app to a file
basic_app = '''
import streamlit as st

# Page configuration (must be first Streamlit command)
st.set_page_config(
    page_title="My First App",
    page_icon="🚀",
    layout="wide"
)

# Title and description
st.title("🚀 My First Streamlit App")
st.markdown("Welcome to Streamlit! This is a basic example.")

# Sidebar
with st.sidebar:
    st.header("Settings")
    name = st.text_input("Your Name", value="Professor Spark")
    enthusiasm = st.slider("Enthusiasm Level", 1, 10, 5)

# Main content
st.subheader(f"Hello, {name}!")
st.write("!" * enthusiasm)

# Columns for layout
col1, col2, col3 = st.columns(3)

with col1:
    st.metric("Temperature", "72°F", "+2°F")

with col2:
    st.metric("Humidity", "45%", "-5%")

with col3:
    st.metric("Wind", "12 mph", "0 mph")

# Interactive elements
if st.button("Click Me!"):
    st.balloons()
    st.success("🎉 You clicked the button!")
'''

# Save to file
with open('/tmp/basic_app.py', 'w') as f:
    f.write(basic_app)

print("Basic app saved to /tmp/basic_app.py")
print("\nTo run it, open a terminal and type:")
print("  streamlit run /tmp/basic_app.py")

### Running Streamlit Apps

Streamlit apps run as web servers. To launch:

```bash
# From terminal
streamlit run app.py

# With custom port
streamlit run app.py --server.port 8501

# For DGX Spark (allow external access)
streamlit run app.py --server.address 0.0.0.0 --server.port 8501
```

---

## Part 2: Multi-Page Applications

### 🧒 ELI5: Multi-Page Apps

> Think of your app like a book. The main page is the cover, and each page in the `pages/` folder is a chapter. Streamlit automatically creates a navigation menu from your chapter files.

### File Structure

```
my_app/
├── Home.py                    # Main entry point (shows first)
├── pages/
│   ├── 1_💬_Chat.py          # Page 1: Chat interface
│   ├── 2_📊_Analytics.py     # Page 2: Analytics dashboard
│   └── 3_⚙️_Settings.py      # Page 3: Settings
├── utils/
│   ├── __init__.py
│   └── helpers.py             # Shared utility functions
└── .streamlit/
    └── config.toml            # Streamlit configuration
```

### Key Rules:
1. The `pages/` folder must be named exactly `pages`
2. Files are sorted alphabetically (use numbers for ordering: `1_Page.py`, `2_Page.py`)
3. Emojis in filenames show in the sidebar navigation
4. Underscores become spaces in the nav (e.g., `1_My_Page.py` → "My Page")

In [ ]:
import os

# Create multi-page app structure
app_dir = '/tmp/streamlit_multipage'
os.makedirs(f'{app_dir}/pages', exist_ok=True)
os.makedirs(f'{app_dir}/.streamlit', exist_ok=True)

# Home.py - Main entry point
home_page = '''
import streamlit as st

st.set_page_config(
    page_title="AI Assistant",
    page_icon="🤖",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Initialize session state (only happens once)
if "messages" not in st.session_state:
    st.session_state.messages = []
if "settings" not in st.session_state:
    st.session_state.settings = {
        "model": "llama3.1:8b",
        "temperature": 0.7,
        "theme": "light"
    }

st.title("🤖 AI Assistant")
st.markdown("""
Welcome to the AI Assistant demo! This multi-page application showcases:

- **💬 Chat**: Conversational AI interface
- **📊 Analytics**: Usage metrics and performance
- **⚙️ Settings**: Configure the assistant

Use the sidebar to navigate between pages.
""")

# Quick stats from session state
col1, col2, col3 = st.columns(3)
col1.metric("Messages", len(st.session_state.messages))
col2.metric("Model", st.session_state.settings["model"].split(":")[0])
col3.metric("Temperature", st.session_state.settings["temperature"])

st.markdown("---")
st.info("👈 Use the sidebar to navigate to different pages.")
'''

with open(f'{app_dir}/Home.py', 'w') as f:
    f.write(home_page)

print("✅ Created Home.py")

In [ ]:
# Chat page
chat_page = '''
import streamlit as st
import time

st.set_page_config(page_title="Chat", page_icon="💬")

st.title("💬 Chat with AI")

# Ensure session state is initialized
if "messages" not in st.session_state:
    st.session_state.messages = []
if "settings" not in st.session_state:
    st.session_state.settings = {"model": "llama3.1:8b", "temperature": 0.7}

# Sidebar: Current settings
with st.sidebar:
    st.subheader("Current Settings")
    st.write(f"Model: {st.session_state.settings['model']}")
    st.write(f"Temperature: {st.session_state.settings['temperature']}")
    
    if st.button("Clear Chat"):
        st.session_state.messages = []
        st.rerun()

# Display chat history
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.write(message["content"])

# Chat input
if prompt := st.chat_input("Ask something..."):
    # Add user message
    st.session_state.messages.append({"role": "user", "content": prompt})
    
    with st.chat_message("user"):
        st.write(prompt)
    
    # Generate response (simulated)
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            time.sleep(1)  # Simulate API call
            
            # In a real app, call your LLM here
            response = f"Echo from {st.session_state.settings['model']}: {prompt}"
            st.write(response)
    
    # Add assistant message
    st.session_state.messages.append({"role": "assistant", "content": response})
'''

with open(f'{app_dir}/pages/1_💬_Chat.py', 'w') as f:
    f.write(chat_page)

print("✅ Created pages/1_💬_Chat.py")

In [ ]:
# Analytics page
analytics_page = '''
import streamlit as st
import random

st.set_page_config(page_title="Analytics", page_icon="📊")

st.title("📊 Analytics")

# Ensure session state is initialized
if "messages" not in st.session_state:
    st.session_state.messages = []

# Calculate metrics from session state
total_messages = len(st.session_state.messages)
user_messages = sum(1 for m in st.session_state.messages if m["role"] == "user")
ai_messages = total_messages - user_messages

# Metrics row
col1, col2, col3, col4 = st.columns(4)
col1.metric("Total Messages", total_messages)
col2.metric("User Messages", user_messages)
col3.metric("AI Responses", ai_messages)
col4.metric("Avg Response Time", "1.2s")

st.markdown("---")

# Chat history
st.subheader("📝 Conversation History")

if not st.session_state.messages:
    st.info("No messages yet. Start a conversation in the Chat page!")
else:
    for i, msg in enumerate(st.session_state.messages):
        role_emoji = "👤" if msg["role"] == "user" else "🤖"
        st.text(f"{role_emoji} {msg['role'].title()}: {msg['content'][:100]}...")

st.markdown("---")

# Simulated charts
st.subheader("📈 Usage Over Time")

# Generate some fake data for the demo
import pandas as pd
import numpy as np

chart_data = pd.DataFrame(
    np.random.randn(20, 3) * 10 + 50,
    columns=["Messages", "Tokens", "Response Time (ms)"]
)

st.line_chart(chart_data)

# Export button
st.markdown("---")
if st.button("📥 Export Chat History"):
    if st.session_state.messages:
        export_text = "\\n".join([f"{m['role']}: {m['content']}" for m in st.session_state.messages])
        st.download_button(
            "Download as TXT",
            export_text,
            file_name="chat_history.txt",
            mime="text/plain"
        )
    else:
        st.warning("No messages to export.")
'''

with open(f'{app_dir}/pages/2_📊_Analytics.py', 'w') as f:
    f.write(analytics_page)

print("✅ Created pages/2_📊_Analytics.py")

In [ ]:
# Settings page
settings_page = '''
import streamlit as st

st.set_page_config(page_title="Settings", page_icon="⚙️")

st.title("⚙️ Settings")

# Ensure session state is initialized
if "settings" not in st.session_state:
    st.session_state.settings = {
        "model": "llama3.1:8b",
        "temperature": 0.7,
        "theme": "light"
    }

st.subheader("Model Configuration")

# Model selection
model = st.selectbox(
    "Select Model",
    ["llama3.1:8b", "llama3.1:70b", "mistral:7b", "codellama:13b"],
    index=["llama3.1:8b", "llama3.1:70b", "mistral:7b", "codellama:13b"].index(
        st.session_state.settings["model"]
    )
)

# Temperature
temperature = st.slider(
    "Temperature",
    min_value=0.0,
    max_value=1.0,
    value=st.session_state.settings["temperature"],
    step=0.1,
    help="Higher = more creative, Lower = more focused"
)

# Max tokens
max_tokens = st.number_input(
    "Max Tokens",
    min_value=100,
    max_value=4000,
    value=1000,
    step=100
)

st.markdown("---")
st.subheader("Appearance")

# Theme (visual only in this demo)
theme = st.radio(
    "Theme",
    ["light", "dark"],
    index=0 if st.session_state.settings["theme"] == "light" else 1
)

st.markdown("---")

# Save button
if st.button("💾 Save Settings", type="primary"):
    st.session_state.settings = {
        "model": model,
        "temperature": temperature,
        "theme": theme
    }
    st.success("✅ Settings saved!")
    st.balloons()

# Show current settings
with st.expander("Current Settings (Debug)"):
    st.json(st.session_state.settings)
'''

with open(f'{app_dir}/pages/3_⚙️_Settings.py', 'w') as f:
    f.write(settings_page)

print("✅ Created pages/3_⚙️_Settings.py")

In [ ]:
# Streamlit config file
config = '''
[theme]
primaryColor = "#007bff"
backgroundColor = "#ffffff"
secondaryBackgroundColor = "#f0f2f6"
textColor = "#262730"
font = "sans serif"

[server]
maxUploadSize = 200
enableCORS = false
'''

with open(f'{app_dir}/.streamlit/config.toml', 'w') as f:
    f.write(config)

print("✅ Created .streamlit/config.toml")
print("\n" + "="*50)
print("Multi-page app created!")
print("\nTo run it:")
print(f"  cd {app_dir}")
print("  streamlit run Home.py")
print("="*50)

---

## Part 3: Session State Deep Dive

### 🧒 ELI5: Session State

> Imagine you're playing a video game. When you pause and unpause, you expect your score, inventory, and position to still be there. That's "state".
>
> In Streamlit, every time you click something, the whole app re-runs (like the game restarting). `st.session_state` is like a save file - it keeps your data between re-runs.
>
> Without it, every click would reset everything!

### Session State Patterns

In [ ]:
# Session state examples
session_state_examples = '''
import streamlit as st

st.title("🔧 Session State Patterns")

# Pattern 1: Initialize with default values
st.header("Pattern 1: Safe Initialization")

# ✅ CORRECT: Check before initializing
if "counter" not in st.session_state:
    st.session_state.counter = 0

# ❌ WRONG: This resets the counter every rerun!
# st.session_state.counter = 0

col1, col2 = st.columns(2)
with col1:
    if st.button("+1"):
        st.session_state.counter += 1
with col2:
    if st.button("-1"):
        st.session_state.counter -= 1

st.metric("Counter", st.session_state.counter)

st.markdown("---")

# Pattern 2: Dictionary-style vs attribute-style
st.header("Pattern 2: Access Styles")

if "user_data" not in st.session_state:
    st.session_state.user_data = {"name": "Unknown", "score": 0}

st.code("""
# Both work:
st.session_state.counter        # Attribute style
st.session_state["counter"]     # Dictionary style

# For complex keys, use dictionary style:
st.session_state["user-data"]   # Dashes not allowed in attribute style
""")

st.markdown("---")

# Pattern 3: Callback functions
st.header("Pattern 3: Callbacks with Session State")

if "form_data" not in st.session_state:
    st.session_state.form_data = ""

def on_input_change():
    """Callback that runs when input changes."""
    # Access the input value from session state (set by key parameter)
    st.session_state.form_data = f"You typed: {st.session_state.my_input}"

st.text_input(
    "Type something",
    key="my_input",  # This creates st.session_state.my_input
    on_change=on_input_change  # Called when value changes
)

st.write(st.session_state.form_data)

st.markdown("---")

# Pattern 4: Clearing state
st.header("Pattern 4: Clearing State")

if st.button("Reset Everything"):
    # Clear specific keys
    for key in ["counter", "user_data", "form_data"]:
        if key in st.session_state:
            del st.session_state[key]
    st.rerun()

# Debug: Show all session state
with st.expander("🔍 View All Session State"):
    st.json(dict(st.session_state))
'''

with open('/tmp/session_state_examples.py', 'w') as f:
    f.write(session_state_examples)

print("Session state examples saved to /tmp/session_state_examples.py")

### Chat History with Session State

The most common use case: maintaining conversation history.

In [ ]:
# Complete chat implementation with session state
chat_with_state = '''
import streamlit as st
import time

st.set_page_config(page_title="Chat Demo", page_icon="💬")

st.title("💬 Chat with Memory")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "assistant", "content": "Hello! How can I help you today?"}
    ]

# Display chat messages
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Accept user input
if prompt := st.chat_input("What's on your mind?"):
    # Add user message to history
    st.session_state.messages.append({"role": "user", "content": prompt})
    
    # Display user message
    with st.chat_message("user"):
        st.markdown(prompt)
    
    # Generate response
    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        full_response = ""
        
        # Simulate streaming response
        response_text = f"I understood: \'{prompt}\'. This is a demo response that streams character by character!"
        
        for char in response_text:
            full_response += char
            message_placeholder.markdown(full_response + "▌")
            time.sleep(0.02)
        
        message_placeholder.markdown(full_response)
    
    # Add assistant response to history
    st.session_state.messages.append({"role": "assistant", "content": full_response})

# Sidebar with controls
with st.sidebar:
    st.subheader("Chat Controls")
    
    st.write(f"Messages: {len(st.session_state.messages)}")
    
    if st.button("🗑️ Clear History"):
        st.session_state.messages = [
            {"role": "assistant", "content": "Hello! How can I help you today?"}
        ]
        st.rerun()
    
    if st.button("📥 Export Chat"):
        chat_export = "\\n\\n".join([
            f"{m['role'].upper()}: {m['content']}" 
            for m in st.session_state.messages
        ])
        st.download_button(
            "Download",
            chat_export,
            file_name="chat_export.txt"
        )
'''

with open('/tmp/chat_with_state.py', 'w') as f:
    f.write(chat_with_state)

print("Chat with state example saved!")

---

## Part 4: Caching Strategies

### 🧒 ELI5: Caching

> Imagine you're asked the same math problem 100 times. The first time, you solve it. The next 99 times, you just remember the answer.
>
> Caching is "remembering" expensive computations so you don't repeat them. In Streamlit:
> - `@st.cache_data`: Remember the RESULT of a function (good for data)
> - `@st.cache_resource`: Remember an OBJECT (good for models, database connections)

### Cache Types Comparison

| Feature | `@st.cache_data` | `@st.cache_resource` |
|---------|------------------|----------------------|
| Purpose | Data (DataFrames, lists, dicts) | Resources (models, connections) |
| Serialization | Yes (copies data) | No (returns same object) |
| Thread-safe | Yes | No (be careful!) |
| Use for | API responses, computations | ML models, DB connections |

In [ ]:
# Caching examples
caching_examples = '''
import streamlit as st
import time

st.title("⚡ Caching Demo")

# @st.cache_data - For data (DataFrames, lists, results)
st.header("1. @st.cache_data - Caching Data")

@st.cache_data  # Cache the result
def expensive_data_computation(n):
    """Simulate an expensive data computation."""
    time.sleep(2)  # Simulate slow computation
    return {"result": n ** 2, "computed_at": time.strftime("%H:%M:%S")}

num = st.number_input("Enter a number", value=5, min_value=1, max_value=100)

start = time.time()
result = expensive_data_computation(num)
elapsed = time.time() - start

st.write(f"Result: {result['result']}")
st.write(f"Computed at: {result['computed_at']}")
st.write(f"⏱️ Took: {elapsed:.2f} seconds (first call ~2s, cached calls ~0s)")

st.markdown("---")

# @st.cache_data with TTL (time to live)
st.header("2. @st.cache_data with TTL")

@st.cache_data(ttl=60)  # Cache expires after 60 seconds
def fetch_api_data():
    """Simulate fetching data from an API."""
    time.sleep(1)
    return {"data": "fresh_data", "timestamp": time.strftime("%H:%M:%S")}

if st.button("Fetch API Data"):
    data = fetch_api_data()
    st.json(data)

st.markdown("---")

# @st.cache_resource - For resources (models, connections)
st.header("3. @st.cache_resource - Caching Resources")

@st.cache_resource  # Load once, reuse everywhere
def load_model():
    """Simulate loading a large ML model."""
    st.write("⏳ Loading model... (only happens once!)")
    time.sleep(3)  # Simulate slow model loading
    
    # In real code:
    # from transformers import pipeline
    # return pipeline("text-generation", model="gpt2")
    
    return {"model": "fake_model", "loaded_at": time.strftime("%H:%M:%S")}

st.info("Click the button to use the model. First click loads it, subsequent clicks reuse it.")

if st.button("Use Model"):
    start = time.time()
    model = load_model()
    elapsed = time.time() - start
    
    st.success(f"Model ready! Loaded at: {model['loaded_at']}")
    st.write(f"⏱️ Took: {elapsed:.2f} seconds")

st.markdown("---")

# Clearing cache
st.header("4. Clearing Cache")

col1, col2 = st.columns(2)

with col1:
    if st.button("Clear Data Cache"):
        st.cache_data.clear()
        st.success("Data cache cleared!")
        
with col2:
    if st.button("Clear Resource Cache"):
        st.cache_resource.clear()
        st.success("Resource cache cleared!")

st.markdown("---")

# Best practices
st.header("📚 Best Practices")
st.markdown("""
1. **Use `@st.cache_data` for:**
   - DataFrames, lists, dictionaries
   - API responses
   - Computation results

2. **Use `@st.cache_resource` for:**
   - ML models (HuggingFace, PyTorch, etc.)
   - Database connections
   - Expensive-to-create objects

3. **Common patterns:**
   ```python
   @st.cache_resource
   def load_llm():
       import ollama
       return ollama.Client()
   
   @st.cache_data(ttl=3600)  # Refresh every hour
   def get_embeddings(text):
       client = load_llm()
       return client.embeddings(model="nomic-embed-text", prompt=text)
   ```
""")
'''

with open('/tmp/caching_examples.py', 'w') as f:
    f.write(caching_examples)

print("Caching examples saved!")

---

## Part 5: Performance Optimization

### Common Performance Issues

1. **Model reloads on every interaction** → Use `@st.cache_resource`
2. **Slow data loading** → Use `@st.cache_data`
3. **UI freezes during computation** → Use spinners and progress bars
4. **Unnecessary reruns** → Use session state wisely

In [ ]:
# Performance optimization patterns
performance_patterns = '''
import streamlit as st
import time

st.title("🚀 Performance Patterns")

# Pattern 1: Lazy Loading
st.header("1. Lazy Loading")

st.markdown("""
Don\'t load everything upfront. Load resources only when needed.
""")

@st.cache_resource
def load_heavy_resource():
    time.sleep(2)
    return "Heavy resource loaded!"

if st.checkbox("I need the heavy resource"):
    with st.spinner("Loading..."):
        resource = load_heavy_resource()
        st.success(resource)

st.markdown("---")

# Pattern 2: Progress Indicators
st.header("2. Progress Indicators")

if st.button("Run Long Process"):
    progress_bar = st.progress(0)
    status_text = st.empty()
    
    for i in range(100):
        time.sleep(0.02)
        progress_bar.progress(i + 1)
        status_text.text(f"Processing: {i+1}%")
    
    status_text.text("Done!")
    st.balloons()

st.markdown("---")

# Pattern 3: Fragment-based Updates (Streamlit 1.33+)
st.header("3. Avoiding Full Reruns")

st.markdown("""
Use `st.fragment` to update only parts of the app (Streamlit 1.33+):

```python
@st.fragment
def my_component():
    # Only this part reruns when interacted with
    if st.button("Click me"):
        st.write("Clicked!")
```
""")

st.markdown("---")

# Pattern 4: Container Updates
st.header("4. Using Containers for Updates")

st.markdown("""
Use `st.empty()` and `st.container()` for dynamic updates:
""")

container = st.container()
button = st.button("Update Container")

if button:
    with container:
        st.write(f"Updated at {time.strftime('%H:%M:%S')}")
        st.metric("Random Value", f"{time.time() % 100:.2f}")

st.markdown("---")

# Pattern 5: Batch Operations
st.header("5. Batch Operations")

st.markdown("""
Instead of multiple small operations, batch them:

```python
# ❌ Slow - multiple state updates
for item in items:
    st.session_state.items.append(process(item))
    st.rerun()  # Reruns after each!

# ✅ Fast - batch the updates
processed = [process(item) for item in items]
st.session_state.items.extend(processed)
st.rerun()  # Single rerun
```
""")
'''

with open('/tmp/performance_patterns.py', 'w') as f:
    f.write(performance_patterns)

print("Performance patterns saved!")

---

## Part 6: Complete Example - AI Agent Playground

Let's build a more complete example that combines everything we've learned.

In [ ]:
# Complete agent playground example
agent_playground = '''
import streamlit as st
import json
import time
from datetime import datetime

st.set_page_config(
    page_title="AI Agent Playground",
    page_icon="🤖",
    layout="wide"
)

# Initialize session state
if "messages" not in st.session_state:
    st.session_state.messages = []
if "tools_enabled" not in st.session_state:
    st.session_state.tools_enabled = {
        "calculator": True,
        "web_search": True,
        "code_executor": False
    }

# Mock tool implementations
def mock_calculator(expression):
    """Mock calculator tool."""
    try:
        result = eval(expression)  # In real code, use a safer evaluator
        return {"tool": "calculator", "input": expression, "output": result}
    except:
        return {"tool": "calculator", "input": expression, "error": "Invalid expression"}

def mock_web_search(query):
    """Mock web search tool."""
    return {
        "tool": "web_search",
        "input": query,
        "output": f"Mock results for: {query}"
    }

def mock_agent_response(message, tools_enabled):
    """Simulate an agent response with tool calls."""
    tool_calls = []
    thinking = ""
    
    # Simulate thinking
    thinking = f"Analyzing user request: \'{message}\'\\n"
    
    # Detect if tools are needed (simple keyword matching)
    if "calculate" in message.lower() and tools_enabled.get("calculator"):
        # Extract a simple expression
        thinking += "User wants a calculation. Using calculator tool.\\n"
        tool_calls.append(mock_calculator("2 + 2"))
    
    if "search" in message.lower() and tools_enabled.get("web_search"):
        thinking += "User wants to search. Using web search tool.\\n"
        tool_calls.append(mock_web_search(message))
    
    thinking += "Formulating response based on available information."
    
    response = f"Based on your request, here\'s my response: {message}"
    
    return {
        "role": "assistant",
        "content": response,
        "thinking": thinking,
        "tool_calls": tool_calls
    }

# Main title
st.title("🤖 AI Agent Playground")

# Sidebar: Tool Configuration
with st.sidebar:
    st.header("🔧 Tool Configuration")
    
    st.session_state.tools_enabled["calculator"] = st.checkbox(
        "📊 Calculator",
        value=st.session_state.tools_enabled["calculator"],
        help="Enable mathematical calculations"
    )
    
    st.session_state.tools_enabled["web_search"] = st.checkbox(
        "🔍 Web Search",
        value=st.session_state.tools_enabled["web_search"],
        help="Enable web search capability"
    )
    
    st.session_state.tools_enabled["code_executor"] = st.checkbox(
        "💻 Code Executor",
        value=st.session_state.tools_enabled["code_executor"],
        help="Enable code execution (disabled by default)"
    )
    
    st.markdown("---")
    
    if st.button("🗑️ Clear Conversation"):
        st.session_state.messages = []
        st.rerun()
    
    st.markdown("---")
    st.caption(f"Messages: {len(st.session_state.messages)}")

# Main chat area
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])
        
        # Show tool calls if present
        if "tool_calls" in message and message["tool_calls"]:
            with st.expander("🔧 Tool Calls", expanded=False):
                for tool in message["tool_calls"]:
                    st.code(json.dumps(tool, indent=2), language="json")
        
        # Show thinking if present
        if "thinking" in message and message["thinking"]:
            with st.expander("💭 Thinking Process", expanded=False):
                st.markdown(message["thinking"])

# Chat input
if prompt := st.chat_input("Ask me anything... (try \'calculate\' or \'search\')"):
    # Add user message
    user_msg = {"role": "user", "content": prompt}
    st.session_state.messages.append(user_msg)
    
    with st.chat_message("user"):
        st.markdown(prompt)
    
    # Generate agent response
    with st.chat_message("assistant"):
        with st.spinner("Thinking..."):
            time.sleep(1)  # Simulate processing
            response = mock_agent_response(prompt, st.session_state.tools_enabled)
        
        st.markdown(response["content"])
        
        if response["tool_calls"]:
            with st.expander("🔧 Tool Calls", expanded=True):
                for tool in response["tool_calls"]:
                    st.code(json.dumps(tool, indent=2), language="json")
        
        if response["thinking"]:
            with st.expander("💭 Thinking Process", expanded=False):
                st.markdown(response["thinking"])
    
    st.session_state.messages.append(response)
'''

with open('/tmp/agent_playground.py', 'w') as f:
    f.write(agent_playground)

print("Agent playground saved to /tmp/agent_playground.py")
print("\nTo run: streamlit run /tmp/agent_playground.py")

---

## ⚠️ Common Mistakes

### Mistake 1: Initializing State Wrong

```python
# ❌ WRONG - Resets every rerun!
st.session_state.counter = 0

# ✅ RIGHT - Only initialize if not present
if "counter" not in st.session_state:
    st.session_state.counter = 0
```

### Mistake 2: Not Caching Expensive Operations

```python
# ❌ WRONG - Model loads on every interaction!
model = load_large_model()

# ✅ RIGHT - Load once, cache forever
@st.cache_resource
def get_model():
    return load_large_model()

model = get_model()
```

### Mistake 3: Blocking the UI

```python
# ❌ WRONG - UI freezes with no feedback
result = slow_function()  # User sees nothing for 30 seconds

# ✅ RIGHT - Show progress
with st.spinner("Processing..."):
    result = slow_function()
```

### Mistake 4: Wrong File Structure for Multi-page Apps

```python
# ❌ WRONG - Files don't appear in navigation
my_app/
├── main.py
└── other_pages/    # Wrong folder name!
    └── page1.py

# ✅ RIGHT - Use exact folder name 'pages'
my_app/
├── Home.py         # Entry point (or main.py, app.py)
└── pages/          # Exact name required
    └── 1_Page.py
```

---

## 🎉 Checkpoint

You've learned:
- ✅ Multi-page Streamlit app structure
- ✅ Session state for persistent data
- ✅ `@st.cache_data` for caching computations
- ✅ `@st.cache_resource` for caching models/resources
- ✅ Performance optimization patterns
- ✅ Building a complete agent playground

---

## 🚀 Challenge (Optional)

Build a **Data Explorer App** with:
1. **Home page**: Upload CSV files (store in session state)
2. **Explore page**: View data, filter, sort (use `@st.cache_data`)
3. **Analyze page**: Show charts and statistics
4. **Settings page**: Configure display preferences

Requirements:
- Uploaded data persists across page navigation
- Analysis results are cached
- Smooth progress indicators for large files

---

## 📖 Further Reading

- [Streamlit Documentation](https://docs.streamlit.io/)
- [Multi-page Apps Guide](https://docs.streamlit.io/library/get-started/multipage-apps)
- [Session State Guide](https://docs.streamlit.io/library/api-reference/session-state)
- [Caching Guide](https://docs.streamlit.io/library/advanced-features/caching)

---

## 🧹 Cleanup

In [ ]:
# Clean up temporary files if needed
import os
import shutil

# List created files
temp_files = [
    '/tmp/basic_app.py',
    '/tmp/session_state_examples.py',
    '/tmp/chat_with_state.py',
    '/tmp/caching_examples.py',
    '/tmp/performance_patterns.py',
    '/tmp/agent_playground.py'
]

print("Created example files:")
for f in temp_files:
    if os.path.exists(f):
        print(f"  ✅ {f}")

print(f"\n📁 Multi-page app: /tmp/streamlit_multipage/")
print("\n💡 Run any example with: streamlit run <filename>")